<a href="https://colab.research.google.com/github/geithelmasri/AAI614_Geith1/blob/main/Week_1_Data_Visualization_using_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 1 Data Visualization using Dash**




In [19]:
!pip install dash dash-bootstrap-components pandas plotly

In [20]:
import dash
# Import dcc and html directly from dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

**Step 3: Load the Dataset**

For this lab, we will use the **Gapminder dataset**, which contains information about life expectancy, GDP, and population by country across several years.

We'll load the data using Pandas and inspect the first few rows.



In [21]:
# prompt: load csv file Sample - Superstore.csv

df = pd.read_csv('Sample - Superstore.csv', encoding='ISO-8859-1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [22]:
# prompt: remove column Order Date, Ship Date, Postal Code, Product ID, Product Name,

df = df.drop(['Order Date', 'Ship Date', 'Postal Code', 'Product ID', 'Product Name'], axis=1)
df.head()

,Row ID,Order ID,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,4,US-2015-108966,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2,0.20,2.5164


This dataset includes columns such as:

**Order ID**

**Ship Mode**

**Customer ID**

**Customer Name**

**Country	City**

**State	Region**

**Category**

**Sub-Category**

**Sales	Quantity**

**Discount**

**Profit**

In [23]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [24]:
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H1("Sales and Profit Analysis"),

    dbc.Row([
        dbc.Col([
            html.Label("Select Region:"),
            dcc.Dropdown(
                id='region-dropdown',
                options=[{'label': i, 'value': i} for i in df['Region'].unique()],
                value=df['Region'].unique()[0] if len(df['Region'].unique()) > 0 else None,
                clearable=False
            ),
        ]),
        dbc.Col([
            html.Label("Select State:"),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': i, 'value': i} for i in df['State'].unique()],
                value=df['State'].unique()[0] if len(df['State'].unique()) > 0 else None,
                clearable=False
            ),
        ]),
        dbc.Col([
            html.Label("Select Category:"),
            dcc.Dropdown(
                id='category-dropdown',
                options=[{'label': i, 'value': i} for i in df['Category'].unique()],
                value=df['Category'].unique()[0] if len(df['Category'].unique()) > 0 else None,
                clearable=False
            ),
        ]),
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='category-profit-graph')
        ], width=6),
        dbc.Col([
            dcc.Graph(id='region-sales-bar-graph')
        ], width=6),
    ]),
])


@app.callback(
    Output('category-profit-graph', 'figure'),
    Input('region-dropdown', 'value'),
    Input('state-dropdown', 'value'),
    Input('category-dropdown', 'value')
)
def update_category_profit_graph(selected_region, selected_state, selected_category):
    filtered_df = df[(df['Region'] == selected_region) & (df['State'] == selected_state) & (df['Category'] == selected_category)]
    fig = px.bar(filtered_df, x='Sub-Category', y='Profit', title=f'Profit by Sub-Category for {selected_category} in {selected_state}, {selected_region}')
    return fig


@app.callback(
    Output('region-sales-bar-graph', 'figure'),
    Input('region-dropdown', 'value'),
    Input('state-dropdown', 'value')
)
def update_region_sales_bar_graph(selected_region, selected_state):
    filtered_df = df[(df['Region'] == selected_region) & (df['State'] == selected_state)]
    avg_sales_by_region = filtered_df.groupby('Region')['Sales'].mean().reset_index()
    fig = px.bar(avg_sales_by_region, x='Region', y='Sales', title=f'Average Sales by Region for {selected_state}, {selected_region}')
    return fig

if __name__ == '__main__':
    #app.run_server(debug=True, mode='inline')
    app.run(debug=True)

<IPython.core.display.Javascript object>